In [3]:
import requests
import pandas as pd
from datetime import datetime, timedelta
import time

def fetch_coinbase_1min_data(symbol_pair="BTC-USD", start_date="2025-05-01", end_date="2025-05-31"):
    base_url = "https://api.exchange.coinbase.com/products"
    endpoint = f"{base_url}/{symbol_pair}/candles"
    
    start_dt = datetime.strptime(start_date, "%Y-%m-%d")
    end_dt = datetime.strptime(end_date, "%Y-%m-%d") + timedelta(days=1)
    
    all_data = []
    current_start = start_dt
    batch_hours = 5  # Coinbase allows max 300 candles per request at 1-min = 5 hours
    batch_count = 0
    
    print(f"Data collection started: {start_date} ~ {end_date}")
    print(f"Symbol: {symbol_pair}, Interval: 1min")
    
    while current_start < end_dt:
        current_end = min(current_start + timedelta(hours=batch_hours), end_dt)
        
        params = {
            "granularity": 60,  # 1 minute
            "start": current_start.isoformat(),
            "end": current_end.isoformat()
        }
        
        try:
            print(f"Batch {batch_count + 1}: {current_start.strftime('%Y-%m-%d %H:%M')} ~ {current_end.strftime('%Y-%m-%d %H:%M')}")
            response = requests.get(endpoint, params=params, timeout=30)
            
            if response.status_code == 200:
                batch_data = response.json()
                if batch_data:
                    all_data.extend(batch_data)
                    print(f"  → {len(batch_data)} candles collected")
                else:
                    print("  → No data found")
                    
            elif response.status_code == 429:
                print("  → Rate limit reached, waiting 60 seconds...")
                time.sleep(60)
                continue
            else:
                print(f"  → API error: {response.status_code}")
                
        except requests.exceptions.RequestException as e:
            print(f"  → Request failed: {e}")
            
        current_start = current_end
        batch_count += 1
        time.sleep(1)
    
    if not all_data:
        print("No data retrieved.")
        return pd.DataFrame()
    
    df = pd.DataFrame(all_data, columns=["timestamp", "low", "high", "open", "close", "volume"])
    df["datetime"] = pd.to_datetime(df["timestamp"], unit="s")
    df = df.sort_values("datetime").reset_index(drop=True)
    df = df[["datetime", "timestamp", "open", "high", "low", "close", "volume"]]
    
    print(f"\nTotal {len(df)} candles collected")
    print(f"Range: {df['datetime'].min()} ~ {df['datetime'].max()}")
    
    return df

def save_to_csv(df, filename="btc_1min_may2025.csv"):
    if df.empty:
        print("No data to save.")
        return
    try:
        df.to_csv(filename, index=False)
        print(f"\nCSV saved: {filename}")
        print(f"Size: {len(df)} rows x {len(df.columns)} columns")
        print("\nPreview:")
        print(df.head(3))
        print("...")
        print(df.tail(3))
    except Exception as e:
        print(f"Failed to save CSV: {e}")

def main():
    print("="*60)
    print("Bitcoin 1-Minute Interval Data Fetcher")
    print("Period: May 1, 2025 ~ May 31, 2025")
    print("="*60)
    
    btc_data = fetch_coinbase_1min_data(
        symbol_pair="BTC-USD",
        start_date="2025-05-01", 
        end_date="2025-05-31"
    )
    
    if not btc_data.empty:
        save_to_csv(btc_data, "btc_1min_may2025.csv")
        print(f"\n=== Summary ===")
        print(f"Open Price: ${btc_data['open'].iloc[0]:,.2f}")
        print(f"Close Price: ${btc_data['close'].iloc[-1]:,.2f}")
        print(f"High Price: ${btc_data['high'].max():,.2f}")
        print(f"Low Price: ${btc_data['low'].min():,.2f}")
        print(f"Avg. Volume: {btc_data['volume'].mean():,.2f}")
    else:
        print("Data collection failed.")

if __name__ == "__main__":
    main()


Bitcoin 1-Minute Interval Data Fetcher
Period: May 1, 2025 ~ May 31, 2025
Data collection started: 2025-05-01 ~ 2025-05-31
Symbol: BTC-USD, Interval: 1min
Batch 1: 2025-05-01 00:00 ~ 2025-05-01 05:00
  → 301 candles collected
Batch 2: 2025-05-01 05:00 ~ 2025-05-01 10:00
  → 301 candles collected
Batch 3: 2025-05-01 10:00 ~ 2025-05-01 15:00
  → 301 candles collected
Batch 4: 2025-05-01 15:00 ~ 2025-05-01 20:00
  → 301 candles collected
Batch 5: 2025-05-01 20:00 ~ 2025-05-02 01:00
  → 301 candles collected
Batch 6: 2025-05-02 01:00 ~ 2025-05-02 06:00
  → 301 candles collected
Batch 7: 2025-05-02 06:00 ~ 2025-05-02 11:00
  → 301 candles collected
Batch 8: 2025-05-02 11:00 ~ 2025-05-02 16:00
  → 301 candles collected
Batch 9: 2025-05-02 16:00 ~ 2025-05-02 21:00
  → 301 candles collected
Batch 10: 2025-05-02 21:00 ~ 2025-05-03 02:00
  → 301 candles collected
Batch 11: 2025-05-03 02:00 ~ 2025-05-03 07:00
  → 301 candles collected
Batch 12: 2025-05-03 07:00 ~ 2025-05-03 12:00
  → 301 candles 

In [1]:
import requests
import pandas as pd
from datetime import datetime, timedelta
import time

def fetch_coinbase_1min_data(symbol_pair="BTC-USD", days=365, end_time=None):
    base_url = "https://api.exchange.coinbase.com/products"
    endpoint = f"{base_url}/{symbol_pair}/candles"

    end = end_time or datetime.utcnow()
    start = end - timedelta(days=days)

    all_data = []
    current_start = start
    batch_hours = 5  # 300 candles per request at 1-min granularity
    batch_count = 0

    print(f"Fetching 1-min OHLCV data from {start.strftime('%Y-%m-%d')} to {end.strftime('%Y-%m-%d')}")
    print(f"Symbol: {symbol_pair}, Interval: 1min\n")

    while current_start < end:
        current_end = min(current_start + timedelta(hours=batch_hours), end)
        params = {
            "granularity": 60,
            "start": current_start.isoformat(),
            "end": current_end.isoformat()
        }

        try:
            print(f"Batch {batch_count + 1}: {current_start.strftime('%Y-%m-%d %H:%M')} ~ {current_end.strftime('%Y-%m-%d %H:%M')}")
            response = requests.get(endpoint, params=params, timeout=30)

            if response.status_code == 200:
                batch_data = response.json()
                if batch_data:
                    all_data.extend(batch_data)
                    print(f"  → {len(batch_data)} candles collected")
                else:
                    print("  → No data found")
            elif response.status_code == 429:
                print("  → Rate limit reached, waiting 60 seconds...")
                time.sleep(60)
                continue
            else:
                print(f"  → API error: {response.status_code}")

        except requests.exceptions.RequestException as e:
            print(f"  → Request failed: {e}")

        current_start = current_end
        batch_count += 1
        time.sleep(1)

    if not all_data:
        print("No data retrieved.")
        return pd.DataFrame()

    df = pd.DataFrame(all_data, columns=["timestamp", "low", "high", "open", "close", "volume"])
    df["datetime"] = pd.to_datetime(df["timestamp"], unit="s")
    df = df.sort_values("datetime").reset_index(drop=True)
    df = df[["datetime", "timestamp", "open", "high", "low", "close", "volume"]]

    print(f"\n✅ Total {len(df)} candles collected")
    print(f"Range: {df['datetime'].min()} ~ {df['datetime'].max()}")
    return df

def save_to_csv(df, filename="btc_1min_past_year.csv"):
    if df.empty:
        print("No data to save.")
        return
    try:
        df.to_csv(filename, index=False)
        print(f"\n📁 CSV saved: {filename}")
        print(f"Size: {len(df)} rows x {len(df.columns)} columns")
        print("\nPreview:")
        print(df.head(3))
        print("...")
        print(df.tail(3))
    except Exception as e:
        print(f"Failed to save CSV: {e}")

def main():
    print("="*60)
    print("Bitcoin 1-Minute OHLCV Data Fetcher (Past 1 Year)")
    print("="*60)

    btc_data = fetch_coinbase_1min_data()

    if not btc_data.empty:
        save_to_csv(btc_data, "btc_1min_past_year.csv")
        print(f"\n=== Summary ===")
        print(f"Open Price: ${btc_data['open'].iloc[0]:,.2f}")
        print(f"Close Price: ${btc_data['close'].iloc[-1]:,.2f}")
        print(f"High Price: ${btc_data['high'].max():,.2f}")
        print(f"Low Price: ${btc_data['low'].min():,.2f}")
        print(f"Avg. Volume: {btc_data['volume'].mean():,.2f}")
    else:
        print("Data collection failed.")

if __name__ == "__main__":
    main()


Bitcoin 1-Minute OHLCV Data Fetcher (Past 1 Year)
Fetching 1-min OHLCV data from 2024-08-07 to 2025-08-07
Symbol: BTC-USD, Interval: 1min

Batch 1: 2024-08-07 19:09 ~ 2024-08-08 00:09


C:\Users\taehy\AppData\Local\Temp\ipykernel_46504\1466875607.py:10: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  end = end_time or datetime.utcnow()


  → 300 candles collected
Batch 2: 2024-08-08 00:09 ~ 2024-08-08 05:09
  → 300 candles collected
Batch 3: 2024-08-08 05:09 ~ 2024-08-08 10:09
  → 300 candles collected
Batch 4: 2024-08-08 10:09 ~ 2024-08-08 15:09
  → 300 candles collected
Batch 5: 2024-08-08 15:09 ~ 2024-08-08 20:09
  → 300 candles collected
Batch 6: 2024-08-08 20:09 ~ 2024-08-09 01:09
  → 300 candles collected
Batch 7: 2024-08-09 01:09 ~ 2024-08-09 06:09
  → 300 candles collected
Batch 8: 2024-08-09 06:09 ~ 2024-08-09 11:09
  → 300 candles collected
Batch 9: 2024-08-09 11:09 ~ 2024-08-09 16:09
  → 300 candles collected
Batch 10: 2024-08-09 16:09 ~ 2024-08-09 21:09
  → 300 candles collected
Batch 11: 2024-08-09 21:09 ~ 2024-08-10 02:09
  → 300 candles collected
Batch 12: 2024-08-10 02:09 ~ 2024-08-10 07:09
  → 300 candles collected
Batch 13: 2024-08-10 07:09 ~ 2024-08-10 12:09
  → 300 candles collected
Batch 14: 2024-08-10 12:09 ~ 2024-08-10 17:09
  → 300 candles collected
Batch 15: 2024-08-10 17:09 ~ 2024-08-10 22:09
